# BrainFlow Band Power Notebook

In [ ]:
import argparse
import time
import brainflow
import numpy as np

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations, WindowFunctions

In [ ]:
# use synthetic board for demo
params = BrainFlowInputParams ()
board_id = BoardIds.SYNTHETIC_BOARD.value
sampling_rate = BoardShim.get_sampling_rate (board_id)
board = BoardShim (board_id, params)
board.prepare_session ()
board.start_stream ()
time.sleep (10)
# get power of 2 values and use FFT freq resolution ~ 1
data = board.get_current_board_data (DataFilter.get_nearest_power_of_two (sampling_rate))
board.stop_stream ()
board.release_session ()
eeg_channels = BoardShim.get_eeg_channels (board_id)
# use first eeg channel for demo
eeg_channel = eeg_channels[0]

In [ ]:
# optional: detrend or subtract mean
mean = np.mean (data[eeg_channel])
data[eeg_channel] = data[eeg_channel] - mean

In [ ]:
psd = DataFilter.get_log_psd (data[eeg_channel], sampling_rate, WindowFunctions.HAMMING.value)
plt.plot (psd[1][:60], psd[0][:60])
plt.show ()

In [ ]:
# calc band power
alpha = DataFilter.get_band_power (psd, 7.0, 13.0)
beta = DataFilter.get_band_power (psd, 14.0, 30.0)
print ("Alpha/Beta Ratio is: %f" % (alpha / beta))